# Projet M2 MIAS : Risque de réidentification (Février 2026)

**Auteur :** [Votre Nom]
**Superviseur :** Pr Emmanuel Chazard

## 1. Objectif du projet
Conformément au sujet, l'objectif est d'imaginer et tester des indicateurs quantitatifs du risque de réidentification (Inférence, Corrélation, Individualisation) sur des bases de données fournies.

Nous analyserons l'évolution du risque en fonction :
1. Du **niveau de mélange** (shuffling) des données.
2. De la **quantité d'informations** disponibles (scénarios de connaissance croissante).
3. De la nature de la base (**Directe** vs **Échantillonnée**).

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

# Configuration graphique
plt.style.use('seaborn-v0_8')

## 2. Chargement des Données

Les fichiers sont stockés dans le répertoire `projets_donnees`.
Nous chargeons :
1. `connaissances_externes.txt` : La base de référence.
2. Tous les fichiers `out_direct_*.txt` et `out_sample_*.txt` trouvés dans le dossier.

**Note sur les variables  :**
Les fichiers contiennent notamment : `id_sejour`, `age` (et variantes), `sexe`, `entree_date` (et variantes), `specialite`, `chirurgie`, `diabete`, etc.

In [ ]:
# Chemin vers le répertoire des données
DATA_DIR = "projets_donnees"

def load_data(directory):
    data = {}
    
    # 1. Chargement de la base de référence
    ref_path = os.path.join(directory, "connaissances_externes.txt")
    if os.path.exists(ref_path):
        # Utilisation de sep=None pour détection auto (csv, tsv, etc.)
        data['reference'] = pd.read_csv(ref_path, sep=None, engine='python')
        print(f"Base référence chargée : {len(data['reference'])} lignes")
    else:
        raise FileNotFoundError(f"Le fichier {ref_path} est introuvable.")

    # 2. Chargement des fichiers variants (out_direct et out_sample)
    # On cherche tous les fichiers commençant par 'out_' dans le dossier
    pattern = os.path.join(directory, "out_*.txt")
    files = glob.glob(pattern)
    
    if not files:
        print("Attention : Aucun fichier 'out_*.txt' trouvé.")

    for filepath in files:
        filename = os.path.basename(filepath)
        name_key = filename.replace('.txt', '') # ex: out_direct_10
        
        try:
            df = pd.read_csv(filepath, sep=None, engine='python')
            data[name_key] = df
            print(f" -> Chargé : {name_key} ({len(df)} lignes)")
        except Exception as e:
            print(f"Erreur lors du chargement de {filename} : {e}")
            
    return data

# Exécution du chargement
datasets = load_data(DATA_DIR)
df_reference = datasets['reference']

# Aperçu
df_reference.head()

## 3. Définition des Indicateurs (Critères G29)

Conformément aux instructions [cite: 30-32], nous définissons des indicateurs expérimentaux pour l'**Individualisation** et l'**Inférence**.

1.  **Taux d'Individualisation (Réidentification)** : Capacité à isoler un enregistrement unique dans la base publiée et à le lier correctement à la base de connaissances via l'`id_sejour` (qui sert de vérité terrain [cite: 28]).
2.  **Taux d'Inférence (Précision)** : Capacité à déduire une valeur sensible (ex: `diabete`) à partir des quasi-identifiants.

In [ ]:
def calculate_risk(df_connaissance, df_publie, quasi_identifiers, sensitive_col='diabete'):
    """
    Calcule les métriques de risque en comparant la base de connaissance (source)
    et la base publiée (attaquée).
    """
    # Vérification que les colonnes existent dans les deux bases
    missing_cols = [c for c in quasi_identifiers if c not in df_publie.columns]
    if missing_cols:
        print(f"Colonnes manquantes dans le fichier publié : {missing_cols}")
        return 0.0, 0.0

    # 1. Isoler les combinaisons uniques dans le jeu publié (Individualisation potentielle)
    counts = df_publie.groupby(quasi_identifiers).size()
    unique_combinations = counts[counts == 1].index
    
    if len(unique_combinations) == 0:
        return 0.0, 0.0
    
    # Filtrer le dataset publié pour ne garder que les uniques
    df_pub_unique = df_publie.set_index(quasi_identifiers)
    df_pub_unique = df_pub_unique[df_pub_unique.index.isin(unique_combinations)].reset_index()
    
    # 2. Tentative de lien avec la base de connaissance (Attaque par jointure)
    merged = pd.merge(
        df_pub_unique, 
        df_connaissance, 
        on=quasi_identifiers, 
        how='inner', 
        suffixes=('_pub', '_know')
    )
    
    if len(merged) == 0:
        return 0.0, 0.0
        
    # --- Indicateur 1 : Individualisation Correcte ---
    # Succès si l'ID caché est le même [cite: 28]
    reid_success = merged[merged['id_sejour_pub'] == merged['id_sejour_know']]
    risk_reid = len(reid_success) / len(df_publie) * 100
    
    # --- Indicateur 2 : Inférence Exacte ---
    # Est-ce que la valeur sensible déduite est la bonne (même si l'ID est faux) ?
    if sensitive_col in df_publie.columns and sensitive_col in df_connaissance.columns:
        # On suppose que l'attaquant prend la valeur de la ligne unique trouvée
        inference_success = merged[merged[f'{sensitive_col}_pub'] == merged[f'{sensitive_col}_know']]
        risk_inference = len(inference_success) / len(merged) * 100
    else:
        risk_inference = 0.0
    
    return risk_reid, risk_inference

## 4. Exécution des Scénarios de "Connaissance Croissante"

Nous définissons 3 scénarios basés sur les variables listées dans le sujet [cite: 10-16] :
1.  **Faible** : Âge imprécis (10 ans), Sexe, Année d'entrée.
2.  **Moyen** : Âge moyen (5 ans), Sexe, Date (Mois), Spécialité.
3.  **Fort** : Âge exact, Sexe, Date exacte (Jour), Spécialité, Mode entrée, Chirurgie.

In [ ]:
# Définition des scénarios selon les variables du PDF [cite: 10-16]
scenarios = {
    '1_Faible': ['age10', 'sexe', 'entree_date_y'],
    '2_Moyen': ['age5', 'sexe', 'entree_date_ym', 'specialite'],
    '3_Fort': ['age', 'sexe', 'entree_date_ymd', 'specialite', 'entree_mode', 'chirurgie']
}

results = []

for name, df_var in datasets.items():
    if name == 'reference': continue
    
    # Extraction du type et du pourcentage depuis le nom du fichier (ex: out_direct_10)
    parts = name.split('_')
    if len(parts) >= 3:
        file_type = 'Direct' if 'direct' in name else 'Sample'
        try:
            shuffle_level = int(parts[-1])
        except ValueError:
            shuffle_level = 0 # Fallback si le nom ne finit pas par un chiffre
    else:
        continue # Fichier mal nommé

    for scen_name, cols in scenarios.items():
        # Calcul des risques pour ce fichier et ce scénario
        r_reid, r_inf = calculate_risk(df_reference, df_var, cols, sensitive_col='diabete')
        
        results.append({
            'Type': file_type,
            'Mélange (%)': shuffle_level,
            'Scénario': scen_name,
            'Nb_Variables': len(cols),
            'Risque_Reid (%)': r_reid,
            'Risque_Inference (%)': r_inf
        })

df_res = pd.DataFrame(results)
display(df_res.head())

## 5. Présentation des Résultats et Discussion

Visualisation de l'évolution des risques en fonction du mélange et des connaissances.

In [ ]:
def plot_results(metric, title):
    if df_res.empty:
        print("Aucun résultat à afficher. Vérifiez le chargement des fichiers.")
        return

    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=df_res, 
        x='Mélange (%)', 
        y=metric, 
        hue='Scénario', 
        style='Type', 
        markers=True, 
        dashes=True
    )
    plt.title(title)
    plt.ylabel(metric)
    plt.ylim(-5, 105)
    plt.show()

# Graphique 1 : Risque d'Individualisation
plot_results('Risque_Reid (%)', 'Risque de Réidentification vs Mélange')

# Graphique 2 : Risque d'Inférence
plot_results('Risque_Inference (%)', "Précision de l'Inférence (Diabète) vs Mélange")

### Discussion des résultats attendus

1.  **Impact du Mélange** : On s'attend à ce que le risque chute drastiquement à mesure que le pourcentage de mélange augmente (les lignes deviennent des composites artificiels qui ne correspondent plus à la référence).
2.  **Impact des Connaissances** : Plus le scénario est "Fort" (plus de variables précises), plus le risque de réidentification est élevé à faible taux de mélange.
3.  **Direct vs Sample** : La méthode "Sample" (tirage avec remise) devrait offrir une meilleure protection (risque plus faible) car elle introduit une incertitude sur la présence de l'individu dans la base[cite: 26].